<a href="https://colab.research.google.com/github/Abhiraj36/HumanEmotionsDetection/blob/main/Human_Emotions_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from PIL import Image
import albumentations as A
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, GlobalAveragePooling2D, Conv2D, MaxPool2D, Dense
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy, TopKCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, CSVLogger, EarlyStopping, LearningRateScheduler
from tensorflow.keras.regularizers import L2,L1



In [ ]:
import zipfile
import os

zip_path = "/content/Human_Emotion_Dataset.zip"  # Update with correct path
extract_path = "/content/Human_Emotion_Dataset"  # Folder where it will be extracted

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")


Dataset extracted successfully!


In [ ]:
dataset_path = "/content/Human_Emotion_Dataset"
train_directory = os.path.join(dataset_path, "EmotionsDataset_Splitted", "data", "train")
validation_directory = os.path.join(dataset_path, "EmotionsDataset_Splitted", "data", "test")


print("Updated train path:", train_directory)
print("Updated validation path:", validation_directory)

CLASS_NAMES = ["angry", "happy", "sad"]



Updated train path: /content/Human_Emotion_Dataset/EmotionsDataset_Splitted/data/train
Updated validation path: /content/Human_Emotion_Dataset/EmotionsDataset_Splitted/data/test


In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_directory,
    labels='inferred',
    label_mode='int',
    image_size=(256, 256),
    batch_size=32,
    shuffle=True,
    seed=99,
    class_names=CLASS_NAMES,
    color_mode='rgb'
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    validation_directory,  # Now pointing to test directory
    labels='inferred',
    label_mode='int',
    image_size=(256, 256),
    batch_size=32,
    shuffle=False,  # Usually, we don't shuffle validation data
    seed=99,
    class_names=CLASS_NAMES,
    color_mode='rgb'
)





Found 2278 files belonging to 3 classes.
Found 569 files belonging to 3 classes.
Datasets loaded successfully! 🚀
